In [3]:
import json
from pathlib import Path
import tiktoken
import pandas as pd

In [2]:
DATA_DIR = Path('data')
DATA_POINTS_FILE = DATA_DIR / 'kradeze_pripady.xlsx'
MODEL = 'gpt-3.5-turbo'

In [4]:
encoding = tiktoken.encoding_for_model(MODEL)

In [5]:
kradeze_df = pd.read_excel(DATA_POINTS_FILE)

In [6]:
kradeze_df['tokens'] = kradeze_df["plny_skutek"].apply(lambda x: len(encoding.encode(x)))

In [38]:
kradeze_df

,Varianta,Podvarianta,Okolnost,ID_slt,plny_skutek,tokens
0,Krádež_v_práci,Nahodilé_příležitostně,NaN,31751427,"Dne 3.1.2017 v době kolem 13:00 hodin v XXX, k...",141
1,Krádež_v_obchodě,pro_získání_prostředků,NaN,31757383,v Brně v OD Tesco v nákupim centru Královo pol...,156
2,Krádež_v_obchodě,spíše_pro_přímou_spotřebu,NaN,31758225,Obviněný v silně podnapilém stavu v prodejně A...,108
3,Krádež_v_obchodě,pro_získání_prostředků,NaN,31781675,v době od 10:30 do 11:30 hodin dne 2. března 2...,1109
4,Krádež_na_návštěvě,naskytla_se_příležitost,NaN,32052032,v hotelu využil zřejmě neuzamčených vstupních ...,218
...,...,...,...,...,...,...
780,Vloupání_do_jiného_objektu,množství_přesahující_osobní_spotřebu,skupinově\nponičení při vstupu,33588849,obžalovaní společně:\nv době od 1.00 hodin do ...,151
781,Krádež_v_obchodě,pro_získání_prostředků,skupinově\n2-4 obdobných útoků,33657041,společně s další obviněnou\nad 1) po předchozí...,469
782,Krádež_v_práci,Nahodilé_příležitostně,skupinově,34097447,dne 3.9.2017 v době od 18:00 hodin do 24:00 ho...,230
783,Krádež_z_volně_příst_místa,v_exteriéru,skupinově\n2-4 obdobných útoků,34194252,1/ v blíže nezjištěné době počátku roku 2016 d...,484


In [39]:
with open('excel_data.json', 'w', encoding='utf-8') as file:
    kradeze_df.to_json(file, orient='records', force_ascii=False, indent=2)

In [40]:
with open('excel_data.json', encoding='utf-8') as file:
    data = json.load(file)

In [41]:
ordered_data = {}
for data_point in sorted(data, key=lambda x: x['tokens']):
    ordered_data[data_point.pop('ID_slt')] = data_point

In [42]:
with open('excel_data.json', 'w', encoding='utf-8') as file:
    json.dump(ordered_data, file, ensure_ascii=False, indent=2)

In [3]:
new_df = pd.read_json('data/kradeze_pripady.json', orient='index', convert_axes=False)

In [4]:
with open('data/kradeze_pripady_3b.json', encoding='utf8') as f:
    theme_map = json.load(f)

In [87]:
candidate2theme = {}
for theme, v in theme_map.items():
    for candidate_theme in v:
        candidate2theme[candidate_theme] = theme

In [8]:
new_df['theme'] = new_df['candidate_theme'].apply(lambda x: candidate2theme[x] if x in candidate2theme else 'Miscellaneous')

NameError: name 'candidate2theme' is not defined

In [5]:
new_df.Varianta.unique()

array(['Krádež_kovů', 'Krádež_v_obchodě', 'Vloupání_do_jiného_objektu',
       'Kapsářství', 'Krádež_z_volně_příst_místa', 'Krádež_kola',
       'Krádež_na_návštěvě', 'Vloupání_do_obydlí', 'Krádež_z_auta',
       'Vykradení_sklepů', 'Ostatní', 'Krádež_v_práci', 'Krádež_energií',
       'Krádež_motorového_vozidla'], dtype=object)

In [81]:
new_df['candidate_theme'].value_counts()

candidate_theme
Workplace Theft                                    35
Shoplifting Food                                   35
Burglary and Theft of Varied Goods                 27
Multi-incident Theft                               23
Shoplifting Mixed Goods                            21
                                                   ..
Shoplifting Personal Care Items and Electronics     1
Burglary and Theft of Electronics and Food          1
Theft of Vehicle and Street Theft                   1
Shoplifting Clothing, Food, and Alcohol             1
Workplace Theft of Tools and Electronics            1
Name: count, Length: 217, dtype: int64

In [7]:
new_df.to_excel('data/transformed_data.xlsx', index=True)

In [5]:
import json

with open('data/kradeze_pripady_bk.json', encoding='utf8') as f:
    data = json.load(f)
        
with open('data/kradeze_pripady.json', encoding='utf8') as f:
    data2 = json.load(f)
    
for key, val in data.items():
    if 'initial_code_0' not in data[key]:
        continue
    init_code = data[key]['initial_code_0']
    data2[key]['initial_code_0'] = init_code
    
with open('data/kradeze_pripady_bk.json', 'w', encoding='utf8') as f:
    json.dump(data2, f, indent=2, ensure_ascii=False)

In [10]:
with open('data/kradeze_pripady_bk.json', encoding='utf8') as f:
    data = json.load(f)
    
    
new_df = pd.read_json('data/kradeze_pripady_bk.json', orient='index', convert_axes=False)
new_df.to_excel('init_codes.xlsx', index=True)

In [11]:
from openpyxl import Workbook


with open('data/kradeze_pripady_3b.json', encoding='utf8') as f:
    data = json.load(f)
    

wb = Workbook()
ws = wb.active
for theme, val in data.items():
    for subtheme in val:
        ws.append([theme, subtheme])
        
wb.save('themes_subthemes.xlsx')

In [11]:
translation_dict = {
    "Krádež_kovů": "Theft of metals",
    "Krádež_v_obchodě": "Theft in a shop",
    "Vloupání_do_jiného_objektu": "Breaking into another object",
    "Kapsářství": "Pickpocketing",
    "Krádež_z_volně_příst_místa": "Theft from an open-access place",
    "Krádež_kola": "Bicycle theft",
    "Krádež_na_návštěvě": "Theft during a visit",
    "Vloupání_do_obydlí": "Breaking into a residence",
    "Krádež_z_auta": "Theft from a car",
    "Vykradení_sklepů": "Robbing of cellars",
    "Ostatní": "Others",
    "Krádež_v_práci": "Theft at work",
    "Krádež_energií": "Theft of energy",
    "Krádež_motorového_vozidla": "Theft of motor vehicles"
}
new_df['Varianta_EN'] = new_df['Varianta'].apply(lambda x: translation_dict[x])

In [12]:
new_df

,Varianta,Podvarianta,Okolnost,plny_skutek,tokens,plny_skutek_short,initial_code,candidate_theme,theme,Varianta_EN
31768883,Krádež_kovů,bez_větších_problémů,None,Obviněný v areálu společnosti v hale odcizil 5...,56,Obviněný v areálu společnosti v hale odcizil 5...,Workplace theft of aluminum briquettes,Workplace Theft,Theft in Commercial Settings,Theft of metals
31784038,Krádež_v_obchodě,pro_získání_prostředků,None,"dne 12.9.2016 kolem 16.00 hod. v , v prodejně ...",62,"dne 12.9.2016 kolem 16.00 hod. v , v prodejně ...",Shoplifting of mannequin wearing clothes,Shoplifting Clothing,Retail Theft,Theft in a shop
32472849,Vloupání_do_jiného_objektu,pro_vlastní_spotřebu,None,"Pachatel se vloupal do provozovny prádelny, od...",68,"Pachatel se vloupal do provozovny prádelny, od...",Burglary and theft of minor items from laundry...,Burglary of Business,Theft in Commercial Settings,Breaking into another object
31757234,Krádež_v_obchodě,spíše_pro_přímou_spotřebu,None,Pachatel odcizil láhev tvrdého alkoholu z prod...,68,Pachatel odcizil láhev tvrdého alkoholu z prod...,Shoplifting of alcohol from grocery store,Shoplifting Alcohol,Retail Theft,Theft in a shop
32480249,Krádež_v_obchodě,pro_získání_prostředků,None,Pachatelka ukradla v prodejně čokoládové bonbo...,68,Pachatelka ukradla v prodejně čokoládové bonbo...,Shoplifting of chocolates from grocery store,Shoplifting Food,Retail Theft,Theft in a shop
...,...,...,...,...,...,...,...,...,...,...
32686725,Krádež_v_práci,systematické_vyvádění,skupinově,"v přesně nezjištěné době, nejméně v období od ...",9874,"v přesně nezjištěné době, nejméně v období od ...",Workplace theft of a large amount of copper ca...,Workplace Large Scale Theft and Resale,Theft in Commercial Settings,Theft at work
32680420,Krádež_z_auta,Systematické,způsobení škody na autě,I.\n1.v dobe od 22:00 hod. dne 12.3.2014 do 06...,10351,I.\n1.v dobe od 22:00 hod. dne 12.3.2014 do 06...,Multiple burglaries and thefts from various ve...,Various Vehicle and Property Thefts,Vehicle Related Theft,Theft from a car
32480454,Krádež_v_obchodě,pro_získání_prostředků,5 a více obdobných útoků,1. v době kolem 12:45 hodin dne 27. října 2016...,10887,1. v době kolem 12:45 hodin dne 27. října 2016...,Multiple shopliftings from supermarkets and hy...,Serial Shoplifting of Mixed Goods,Retail Theft,Theft in a shop
32500614,Krádež_motorového_vozidla,Systematické,skupinové,2. v době od 20:20 hodin dne 3. listopadu 2011...,13817,2. v době od 20:20 hodin dne 3. listopadu 2011...,Multiple burglary and thefts from company prem...,Workplace Theft of Electronics,Vehicle Related Theft,Theft of motor vehicles


In [1]:
with open('excel_data.json', 'w', encoding='utf-8') as file:
    new_df.to_json(file, orient='index', force_ascii=False, indent=2)

NameError: name 'new_df' is not defined

In [4]:
from sklearn.metrics import classification_report


new_df = pd.read_json('data/kradeze_pripady.json', orient='index', convert_axes=False)

print(classification_report(new_df['Varianta_EN'], new_df['Varianta_EN_gpt4']))

                                 precision    recall  f1-score   support

                  Bicycle theft       0.71      0.74      0.73        27
      Breaking into a residence       0.27      0.52      0.36        31
           Breaking into a shop       0.00      0.00      0.00         0
   Breaking into another object       0.80      0.35      0.49       137
                         Others       0.32      0.23      0.27        26
                  Pickpocketing       0.75      0.68      0.71        60
             Robbing of cellars       0.20      0.14      0.17         7
                  Theft at work       0.64      0.71      0.68        73
           Theft during a visit       0.57      0.75      0.65        53
               Theft from a car       0.65      0.70      0.68        37
              Theft from a shop       0.00      0.00      0.00         0
Theft from an open-access place       0.30      0.21      0.25        42
                Theft in a shop       0.87      0.

c:\Users\jarom\PycharmProjects\typical-crimes\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jarom\PycharmProjects\typical-crimes\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jarom\PycharmProjects\typical-crimes\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
gold_labels, pred_labels = [], []
for gold, pred in zip(new_df['Varianta_EN'], new_df['Varianta_EN_gpt4_multi']):
    if gold in pred:
        pred = gold
    else:
        pred = pred[0]
    gold_labels.append(gold)
    pred_labels.append(pred)

print(classification_report(gold_labels, pred_labels))

                                 precision    recall  f1-score   support

                  Bicycle theft       0.90      0.96      0.93        27
  Breaking into a motor vehicle       0.00      0.00      0.00         0
      Breaking into a residence       0.47      0.71      0.56        31
   Breaking into another object       0.89      0.67      0.77       137
                         Others       0.79      0.73      0.76        26
                  Pickpocketing       0.88      0.87      0.87        60
             Robbing of cellars       0.57      0.57      0.57         7
                  Theft at work       0.85      0.86      0.86        73
           Theft during a visit       0.70      0.91      0.79        53
               Theft from a car       0.79      0.84      0.82        37
         Theft from a residence       0.00      0.00      0.00         0
              Theft from a work       0.00      0.00      0.00         0
Theft from an open-access place       0.55      0.

c:\Users\jarom\PycharmProjects\typical-crimes\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jarom\PycharmProjects\typical-crimes\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jarom\PycharmProjects\typical-crimes\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
new_df

,Varianta,Podvarianta,Okolnost,plny_skutek,tokens,plny_skutek_short,initial_code,candidate_theme,theme,Varianta_EN,Varianta_EN_gpt4
31768883,Krádež_kovů,bez_větších_problémů,None,Obviněný v areálu společnosti v hale odcizil 5...,56,Obviněný v areálu společnosti v hale odcizil 5...,Workplace theft of aluminum briquettes,Workplace Theft,Theft in Commercial Settings,Theft of metals,Theft at work
31784038,Krádež_v_obchodě,pro_získání_prostředků,None,"dne 12.9.2016 kolem 16.00 hod. v , v prodejně ...",62,"dne 12.9.2016 kolem 16.00 hod. v , v prodejně ...",Shoplifting of mannequin wearing clothes,Shoplifting Clothing,Retail Theft,Theft in a shop,Theft in a shop
32472849,Vloupání_do_jiného_objektu,pro_vlastní_spotřebu,None,"Pachatel se vloupal do provozovny prádelny, od...",68,"Pachatel se vloupal do provozovny prádelny, od...",Burglary and theft of minor items from laundry...,Burglary of Business,Theft in Commercial Settings,Breaking into another object,Breaking into another object
31757234,Krádež_v_obchodě,spíše_pro_přímou_spotřebu,None,Pachatel odcizil láhev tvrdého alkoholu z prod...,68,Pachatel odcizil láhev tvrdého alkoholu z prod...,Shoplifting of alcohol from grocery store,Shoplifting Alcohol,Retail Theft,Theft in a shop,Theft in a shop
32480249,Krádež_v_obchodě,pro_získání_prostředků,None,Pachatelka ukradla v prodejně čokoládové bonbo...,68,Pachatelka ukradla v prodejně čokoládové bonbo...,Shoplifting of chocolates from grocery store,Shoplifting Food,Retail Theft,Theft in a shop,Theft in a shop
...,...,...,...,...,...,...,...,...,...,...,...
32686725,Krádež_v_práci,systematické_vyvádění,skupinově,"v přesně nezjištěné době, nejméně v období od ...",9874,"v přesně nezjištěné době, nejméně v období od ...",Workplace theft of a large amount of copper ca...,Workplace Large Scale Theft and Resale,Theft in Commercial Settings,Theft at work,Theft at work
32680420,Krádež_z_auta,Systematické,způsobení škody na autě,I.\n1.v dobe od 22:00 hod. dne 12.3.2014 do 06...,10351,I.\n1.v dobe od 22:00 hod. dne 12.3.2014 do 06...,Multiple burglaries and thefts from various ve...,Various Vehicle and Property Thefts,Vehicle Related Theft,Theft from a car,Breaking into a residence
32480454,Krádež_v_obchodě,pro_získání_prostředků,5 a více obdobných útoků,1. v době kolem 12:45 hodin dne 27. října 2016...,10887,1. v době kolem 12:45 hodin dne 27. října 2016...,Multiple shopliftings from supermarkets and hy...,Serial Shoplifting of Mixed Goods,Retail Theft,Theft in a shop,Theft in a shop
32500614,Krádež_motorového_vozidla,Systematické,skupinové,2. v době od 20:20 hodin dne 3. listopadu 2011...,13817,2. v době od 20:20 hodin dne 3. listopadu 2011...,Multiple burglary and thefts from company prem...,Workplace Theft of Electronics,Vehicle Related Theft,Theft of motor vehicles,Breaking into a residence
